# DB2-Salesforce connector: Citation information updates

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Record_ID__c'
object_id = 'nanoHUB_citations__c'

In [2]:
import sys
import os
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import datetime

## Obtain tool information from DB2 

In [3]:
# Basic tool information
sql_query = "select *, c.id as c_id from jos_citations as c \
            left join jos_citations_types as t \
            on c.type = t.id \
            where uid=2143"

# display
print(sql_query)

select *, c.id as c_id from jos_citations as c             left join jos_citations_types as t             on c.type = t.id             where uid=2143


In [4]:
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

df = pd.read_sql_query(sql_query, nanohub_db)

nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [5]:
# convert and clean some columns, so we can, for example, sort values
df['date_publish'] = pd.to_datetime(df['date_publish'], errors='coerce') 
df['created'] = pd.to_datetime(df['created'], errors='coerce') 
df['date_submit'] = pd.to_datetime(df['date_submit'], errors='coerce') 
df['date_accept'] = pd.to_datetime(df['date_accept'], errors='coerce') 

In [6]:
# display
display(df.head(1))
display('size='+str(df.shape))

,id,uid,affiliated,fundedby,created,address,author,booktitle,chapter,cite,...,custom2,custom3,custom4,pdf_url,id,type,type_title,type_desc,type_export,c_id
0,767,2143,1,0,2011-01-24,,Hoon Ryu{{14414}};Sunhee Lee{{13570}};B. Weber...,2010 Silicon Nanoelectronics Workshop,,,...,None,None,None,https://nanohub.org/nanoHUBCitations/citmanage...,7,conference,Conference,None,None,767


'size=(2517, 68)'

## Match data with Salesforce format

In [7]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here

df_sf['Record_ID__c']      = df['c_id']
df_sf['URL__c']            = df['url']
df_sf['NCN_Affiliated__c'] = df['affiliated'].apply(lambda x: 1 if x==1 else 0)
df_sf['Author_List__c']    = df['author']
df_sf['Journal__c']        = df['journal']
df_sf['Year__c']           = df['year']
df_sf['Type__c']           = df['ref_type']
df_sf['Notes__c']          = df['notes']
df_sf['Name']              = df['title'].apply(lambda x: x[:70])
df_sf['Full_title__c']     = df['title']
df_sf['Format__c']         = df['type_title']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Record_ID__c,767,1279
URL__c,http://ieeexplore.ieee.org/stamp/stamp.jsp?tp=...,http://prb.aps.org/abstract/PRB/v86/i20/e205405
NCN_Affiliated__c,1,1
Author_List__c,Hoon Ryu{{14414}};Sunhee Lee{{13570}};B. Weber...,Daniel Kiracofe{{30551}};Arvind Raman{{10597}}
Journal__c,2010 Silicon Nanoelectronics Workshop,Physical Review B
Year__c,2010,2012
Type__c,"R,N","R,N"
Notes__c,{{IEEE}}\r\nLKZ - NCN acknowledged for extensi...,{{APS}}\r\n
Name,Quantum transport in ultra-scaled phosphorous-...,Nonlinear Dynamics Of The Atomic Force Microsc...
Full_title__c,Quantum transport in ultra-scaled phosphorous-...,Nonlinear Dynamics Of The Atomic Force Microsc...


## To Salesforce Sales Cloud CRM

In [8]:
# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

In [9]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2CyUAAU
hello


[Success] CSV upload successful. Job ID = 7505w00000b2CyUAAU


[Success] Closing job successful. Job ID = 7505w00000b2CyUAAU


In [10]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b2CyUAAU',
 'operation': 'upsert',
 'object': 'nanoHUB_citations__c',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-25T11:04:35.000+0000',
 'systemModstamp': '2021-10-25T11:04:37.000+0000',
 'state': 'InProgress',
 'externalIdFieldName': 'Record_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [11]:
from pprint import pprint
pprint(db_s.check_bulk_failed_results())

''
